# Getting Started- Load Python Packages

## Install all the packages through FinRL library


In [45]:
!pip install wrds
!pip install swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/6p/wq3sknmj60d01hr19w3yktxm0000gn/T/pip-req-build-uyng47v3
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/6p/wq3sknmj60d01hr19w3yktxm0000gn/T/pip-req-build-uyng47v3
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit db6bf1667ad1b347c59fb4805e3de929d88b22ae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/quantopian/pyfolio.git to /private/var/folders/6p/wq3sknmj60d01hr19w3yktxm0000gn/T/pip-install-d65d15n7/pyfolio_3833377d10044181bd940c3f565fa106
  Running command git clone --filter=blob:none --quiet https://github.com/quantopian/pyfolio.git /private/var/folders/6p/wq3sknmj60d01hr19w3yktxm0000gn/T/pip-install-d65d15n7/pyfolio_3833377d10044181bd940c3f565fa106
  Resolved https://github.com/qu

## Import Packages

In [46]:
import warnings
warnings.filterwarnings("ignore")

In [47]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

In [48]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

# Load Data

In [49]:
# TRAIN_START_DATE = "2020-01-03"
# TRAIN_END_DATE = "2022-01-28"
# TEST_START_DATE = "2022-02-07"
# TEST_END_DATE = "2023-02-06"

TRAIN_START_DATE = "2010-01-01"
TRAIN_END_DATE = "2019-12-31"
TEST_START_DATE = "2020-01-01"
TEST_END_DATE = "2023-03-01"

In [50]:
ticker_universe = [
  "AAPL",
  "MSFT",
  "GOOG",
  "AMZN",
  # "BRK.B",
  "TSLA",
  "NVDA",
  "META",
  "XOM",
  "UNH",
  "JPM",
  "V",
  "JNJ",
  "WMT",
  "MA",
  "PG",
  "CVX",
  "HD",
  "LLY",
  "ABBV",
  "BAC",
  "MRK",
  "AVGO",
  "KO",
  "ORCL",
  "PEP",
  "PFE",
  "TMO",
  "COST",
  "CSCO",
  "MCD",
  "NKE",
  "CRM",
  "DIS",
  "DHR",
  "ABT",
  "LIN",
  "WFC",
  "TMUS",
  "ACN",
  "MS",
  "VZ",
  "UPS",
  "TXN",
  "ADBE",
  "CMCSA",
  "PM",
  "NEE",
  "BMY",
  "RTX",
  "COP",
]

In [51]:
len(ticker_universe)

50

In [52]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = ticker_universe).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Process Data

In [55]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed

Successfully added technical indicators
Successfully added turbulence index


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2010-01-04,7.622500,7.660714,7.585000,6.505280,493729600,AAPL,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.505280,6.505280,0.000000
1,2010-01-04,26.000362,26.177889,25.870815,19.433031,10829095,ABT,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,19.433031,19.433031,0.000000
2,2010-01-04,41.520000,42.200001,41.500000,32.836601,3650100,ACN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,32.836601,32.836601,0.000000
3,2010-01-04,36.650002,37.299999,36.650002,37.090000,4710200,ADBE,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,37.090000,37.090000,0.000000
4,2010-01-04,6.812500,6.830500,6.657000,6.695000,151998000,AMZN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.695000,6.695000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155612,2023-02-28,220.000000,221.770004,219.500000,219.940002,5385400,V,1,-0.503228,234.738237,217.596701,50.986160,-93.484623,11.525054,225.478670,218.383917,42.604962
155613,2023-02-28,38.700001,38.970001,38.549999,38.810001,16685300,VZ,1,-0.369096,42.250544,38.079457,46.456076,-148.233860,23.267346,40.217333,39.341519,42.604962
155614,2023-02-28,46.669998,47.029999,46.540001,46.770000,18627200,WFC,1,0.467675,48.731290,45.822002,55.524648,23.856444,14.846140,46.336901,44.237055,42.604962
155615,2023-02-28,141.000000,142.649994,140.949997,142.130005,6018000,WMT,1,-0.138707,147.737118,139.199884,48.541128,-41.652048,7.250607,142.988334,144.336602,42.604962


In [56]:
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2010-01-04,7.622500,7.660714,7.585000,6.505280,493729600,AAPL,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.505280,6.505280,0.000000
1,2010-01-04,26.000362,26.177889,25.870815,19.433031,10829095,ABT,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,19.433031,19.433031,0.000000
2,2010-01-04,41.520000,42.200001,41.500000,32.836601,3650100,ACN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,32.836601,32.836601,0.000000
3,2010-01-04,36.650002,37.299999,36.650002,37.090000,4710200,ADBE,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,37.090000,37.090000,0.000000
4,2010-01-04,6.812500,6.830500,6.657000,6.695000,151998000,AMZN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.695000,6.695000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155612,2023-02-28,220.000000,221.770004,219.500000,219.940002,5385400,V,1,-0.503228,234.738237,217.596701,50.986160,-93.484623,11.525054,225.478670,218.383917,42.604962
155613,2023-02-28,38.700001,38.970001,38.549999,38.810001,16685300,VZ,1,-0.369096,42.250544,38.079457,46.456076,-148.233860,23.267346,40.217333,39.341519,42.604962
155614,2023-02-28,46.669998,47.029999,46.540001,46.770000,18627200,WFC,1,0.467675,48.731290,45.822002,55.524648,23.856444,14.846140,46.336901,44.237055,42.604962
155615,2023-02-28,141.000000,142.649994,140.949997,142.130005,6018000,WMT,1,-0.138707,147.737118,139.199884,48.541128,-41.652048,7.250607,142.988334,144.336602,42.604962


In [57]:
len(processed.tic.values)

155617

In [58]:
dates = processed.date.unique().tolist()
for idx, date in enumerate(dates):
    processed.loc[processed.date == date, 'day'] = idx
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2010-01-04,7.622500,7.660714,7.585000,6.505280,493729600,AAPL,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.505280,6.505280,0.000000
1,2010-01-04,26.000362,26.177889,25.870815,19.433031,10829095,ABT,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,19.433031,19.433031,0.000000
2,2010-01-04,41.520000,42.200001,41.500000,32.836601,3650100,ACN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,32.836601,32.836601,0.000000
3,2010-01-04,36.650002,37.299999,36.650002,37.090000,4710200,ADBE,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,37.090000,37.090000,0.000000
4,2010-01-04,6.812500,6.830500,6.657000,6.695000,151998000,AMZN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.695000,6.695000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155612,2023-02-28,220.000000,221.770004,219.500000,219.940002,5385400,V,3310,-0.503228,234.738237,217.596701,50.986160,-93.484623,11.525054,225.478670,218.383917,42.604962
155613,2023-02-28,38.700001,38.970001,38.549999,38.810001,16685300,VZ,3310,-0.369096,42.250544,38.079457,46.456076,-148.233860,23.267346,40.217333,39.341519,42.604962
155614,2023-02-28,46.669998,47.029999,46.540001,46.770000,18627200,WFC,3310,0.467675,48.731290,45.822002,55.524648,23.856444,14.846140,46.336901,44.237055,42.604962
155615,2023-02-28,141.000000,142.649994,140.949997,142.130005,6018000,WMT,3310,-0.138707,147.737118,139.199884,48.541128,-41.652048,7.250607,142.988334,144.336602,42.604962


In [59]:
processed.index = processed['day']
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
day,,,,,,,,,,,,,,,,,
0,2010-01-04,7.622500,7.660714,7.585000,6.505280,493729600,AAPL,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.505280,6.505280,0.000000
0,2010-01-04,26.000362,26.177889,25.870815,19.433031,10829095,ABT,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,19.433031,19.433031,0.000000
0,2010-01-04,41.520000,42.200001,41.500000,32.836601,3650100,ACN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,32.836601,32.836601,0.000000
0,2010-01-04,36.650002,37.299999,36.650002,37.090000,4710200,ADBE,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,37.090000,37.090000,0.000000
0,2010-01-04,6.812500,6.830500,6.657000,6.695000,151998000,AMZN,0,0.000000,6.526806,6.495000,100.000000,66.666667,100.000000,6.695000,6.695000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310,2023-02-28,220.000000,221.770004,219.500000,219.940002,5385400,V,3310,-0.503228,234.738237,217.596701,50.986160,-93.484623,11.525054,225.478670,218.383917,42.604962
3310,2023-02-28,38.700001,38.970001,38.549999,38.810001,16685300,VZ,3310,-0.369096,42.250544,38.079457,46.456076,-148.233860,23.267346,40.217333,39.341519,42.604962
3310,2023-02-28,46.669998,47.029999,46.540001,46.770000,18627200,WFC,3310,0.467675,48.731290,45.822002,55.524648,23.856444,14.846140,46.336901,44.237055,42.604962


# Split Data

In [60]:
train = data_split(processed, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed, TEST_START_DATE,TEST_END_DATE)
print(len(train))
print(len(trade))

118205
37365


In [61]:
train.tail()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
2514,2019-12-30,189.309998,189.479996,187.119995,183.788254,4833600,V,2514,2.095544,186.318960,175.283878,58.536635,122.507558,17.378878,179.752649,176.410375,18.750532
2514,2019-12-30,61.650002,61.689999,61.090000,52.367279,6765400,VZ,2514,0.289575,53.050522,51.418829,55.330145,71.344225,12.694190,51.845407,51.607820,18.750532
2514,2019-12-30,54.169998,54.259998,53.500000,49.354500,10900500,WFC,2514,0.188931,50.314297,48.621216,54.957156,-9.987022,8.803324,49.553697,48.198697,18.750532
2514,2019-12-30,119.800003,119.809998,119.129997,113.812134,2945800,WMT,2514,0.214172,115.407775,112.420255,53.085354,-1.424315,11.864995,113.689144,113.313073,18.750532
2514,2019-12-30,70.089996,70.440002,69.400002,57.856258,12689400,XOM,2514,0.207807,58.968862,56.677856,50.323848,62.840157,3.070529,57.620040,57.333975,18.750532


In [62]:
trade.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2020-01-02,74.059998,75.150002,73.797501,73.449379,135480400,AAPL,2516,2.139196,73.369910,63.179187,76.464893,176.819703,58.795784,67.057852,63.724705,49.553217
0,2020-01-02,86.059998,86.959999,85.800003,82.532570,4969000,ABT,2516,0.695488,83.927528,79.892275,56.840558,54.104496,5.840768,81.350194,79.744931,49.553217
0,2020-01-02,210.860001,211.919998,208.809998,200.733612,2431100,ACN,2516,3.617984,205.645955,188.886606,64.351844,98.755923,37.063893,194.856652,186.940185,49.553217
0,2020-01-02,330.000000,334.480011,329.170013,334.429993,1990100,ADBE,2516,8.903298,342.941103,296.064899,69.747878,109.325637,45.346731,314.243000,296.655334,49.553217
0,2020-01-02,93.750000,94.900497,93.207497,94.900497,80580000,AMZN,2516,1.306534,94.411533,84.826866,62.204592,215.624114,42.491162,89.280133,88.785950,49.553217


# Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

In [63]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 47, State Space: 471


In [64]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [65]:
e_train_gym = StockTradingEnv(df = train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()

# Train Agents

In [66]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

### Agent 1: A2C

In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

### Agent 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

### Agent 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

In [ ]:

trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

### Agent 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

In [ ]:

trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

# In-sample Performance

In [ ]:
data_risk_indicator = processed[(processed.date<TRAIN_END_DATE) & (processed.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:

insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()


In [ ]:

insample_risk_indicator.turbulence.quantile(0.996)

# Trading Out of Sample Performance

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)

In [ ]:
trade.head()

In [ ]:
trained_moedl = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

In [ ]:
trained_moedl = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

In [ ]:
trained_moedl = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

In [ ]:
print(
    "======A2C Validation from: ",
    TEST_START_DATE,
    "to ",
    TEST_END_DATE,
)
val_env_a2c = DummyVecEnv(
    [
        lambda: StockTradingEnv(
            df=validation,
            stock_dim=self.stock_dim,
            hmax=self.hmax,
            initial_amount=self.initial_amount,
            num_stock_shares=[0] * self.stock_dim,
            buy_cost_pct=[self.buy_cost_pct] * self.stock_dim,
            sell_cost_pct=[self.sell_cost_pct] * self.stock_dim,
            reward_scaling=self.reward_scaling,
            state_space=self.state_space,
            action_space=self.action_space,
            tech_indicator_list=self.tech_indicator_list,
            turbulence_threshold=turbulence_threshold,
            iteration=i,
            model_name="A2C",
            mode="validation",
            print_verbosity=self.print_verbosity,
        )
    ]
)
val_obs_a2c = val_env_a2c.reset()
self.DRL_validation(
    model=model_a2c,
    test_data=validation,
    test_env=val_env_a2c,
    test_obs=val_obs_a2c,
)
sharpe_a2c = self.get_validation_sharpe(i, model_name="A2C")
print("A2C Sharpe Ratio: ", sharpe_a2c)

# Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [26]:
unique_trade_date = processed[(processed.date > TEST_START_DATE)&(processed.date <= TEST_END_DATE)].date.unique()

In [33]:
rebalance_window = 60
validation_window = 60

In [ ]:
df_trade_date = pd.DataFrame({'datadate':unique_trade_date})

df_account_value=pd.DataFrame()
for i in range(rebalance_window+validation_window, len(unique_trade_date)+1,rebalance_window):
    temp = pd.read_csv('results/account_value_trade_{}_{}.csv'.format('ensemble',i))
    df_account_value = df_account_value.append(temp,ignore_index=True)
sharpe=(252**0.5)*df_account_value.account_value.pct_change(1).mean()/df_account_value.account_value.pct_change(1).std()
print('Sharpe Ratio: ',sharpe)
df_account_value=df_account_value.join(df_trade_date[validation_window:].reset_index(drop=True))

In [ ]:
df_account_value.head()

In [ ]:
%matplotlib inline
df_account_value.account_value.plot()

## BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^GSPC", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

## BackTestPlot

In [ ]:
print("==============Compare to SP500===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
              baseline_ticker = '^GSPC', 
              baseline_start = df_account_value.loc[0,'date'],
              baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])